<a href="https://colab.research.google.com/github/multitude00999/MSAI_337_project/blob/main/human_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install evaluate
!pip install sentence_transformers
!pip install rouge_score
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.9 MB/s eta 0:00

In [ ]:
from datasets import load_dataset,get_dataset_split_names,load_from_disk
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np
from transformers import pipeline
import os
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
import pickle
from typing import List
import yaml
import time
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import pandas as pd

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
dataset = load_dataset('mteb/summeval')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/mteb___parquet/mteb--summeval-d1ea838b14528bd7/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['machine_summaries', 'human_summaries', 'relevance', 'coherence', 'fluency', 'consistency', 'text', 'id'],
        num_rows: 100
    })
})

In [ ]:
print(len(dataset['test']['machine_summaries'][0]))
print(len(dataset['test']['human_summaries'][0]))
print(len(dataset['test']['relevance'][0]))
print(len(dataset['test']['coherence'][0]))
print(len(dataset['test']['fluency'][0]))
print(len(dataset['test']['consistency'][0]))
print(len(dataset['test']['text'][0]))

16
11
16
16
16
16
2335


In [ ]:
machine_summary = []
reference_summary = []
relevance_scores = []
coherence_scores = []
fluency_scores = []
consistency_scores = []

for row in dataset['test']:
  machine_summary.append(row['machine_summaries'][0])
  reference_summary.append(row['human_summaries'][0])
  relevance_scores.append(row['relevance'][0])
  coherence_scores.append(row['coherence'][0])
  fluency_scores.append(row['fluency'][0])
  consistency_scores.append(row['consistency'][0])

df = pd.DataFrame()
df['machine_summary'] = machine_summary
df['reference_summary'] = reference_summary
df['relevance_scores'] = relevance_scores
df['coherence_scores'] = coherence_scores
df['consistency_scores'] = consistency_scores
df['fluency_scores'] = fluency_scores

In [ ]:
df['relevance_scores'] = df['relevance_scores']/5
df['coherence_scores'] = df['coherence_scores']/5
df['consistency_scores'] = df['consistency_scores']/5
df['fluency_scores'] = df['fluency_scores']/5

In [ ]:
df.head()

,machine_summary,reference_summary,relevance_scores,coherence_scores,consistency_scores,fluency_scores
0,"donald sterling , nba team last year . sterlin...",V. Stiviano must pay back $2.6 million in gift...,0.333333,0.266667,0.200000,0.200000
1,north pacific gray whale has earned a spot in ...,"The whale, Varvara, swam a round trip from Rus...",0.466667,0.266667,0.266667,0.200000
2,russian fighter jet intercepted a u.s. reconna...,The incident occurred on April 7 north of Pola...,0.800000,0.666667,1.000000,0.733333
3,michael barnett captured the fire on interstat...,Country band Lady Antebellum's bus caught fire...,0.400000,0.400000,0.466667,0.533333
4,deep reddish color caught seattle native tim d...,Smoke from massive fires in Siberia created fi...,0.333333,0.333333,0.400000,1.000000


In [ ]:
df.to_csv('correlation.csv')

In [ ]:
predicted_summary = list(df['machine_summary'])
ground_truths = list(df['reference_summary'])

In [ ]:
def loadModel(model_path):
  return SentenceTransformer(model_path)

def getSentences(text):
  return sent_tokenize(text)

def getEmbeddings(model, sentences):
  embeddings = model.encode(sentences)
  return embeddings

def getSimilarityMatrix(generated_embs, reference_embs):
  return cosine_similarity(generated_embs, reference_embs)

# def getScore(sim_mat):
  
#   # get maximum along reference text axis
#   scores = np.max(sim_mat, axis = 0)

#   # calculate mean
#   final_score = np.mean(scores)

#   return final_score

def getScore(sim_mat):
  
  # get maximum along reference text axis
  scores_recall = np.max(sim_mat, axis = 0)
  
  # calculate mean
  mean_score_recall = np.mean(scores_recall)

  # get maximum along reference text axis
  scores_precision = np.max(sim_mat, axis = 1)
  
  # calculate mean
  mean_score_precision = np.mean(scores_precision)

  # calculate f1 score
  mean_score_f1 = 2*(mean_score_precision)*(mean_score_recall)/(mean_score_precision + mean_score_recall)

  scores = {'precision': mean_score_precision, 'recall': mean_score_recall, 'F1': mean_score_f1}
  return scores

def calcSBERTScore(model, generated_text, reference_text, debug = False):

  # get sentences
  generated_sents = getSentences(generated_text)
  reference_sents = getSentences(reference_text)

  # get embeddings
  generated_embs = getEmbeddings(model, generated_sents)
  reference_embs = getEmbeddings(model, reference_sents)

  # calculate pairwise cosine similarity
  sim_mat = cosine_similarity(generated_embs, reference_embs)

  if debug:
    print("sim mat shape", sim_mat.shape)
    print(sim_mat)
  # get score
  score = getScore(sim_mat)
  
  return score

In [ ]:
print(len(predicted_summary),len(ground_truths))

100 100


In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
# modelsbert_path = 'paraphrase-MiniLM-L6-v2'
modelsbert_path = "multi-qa-mpnet-base-dot-v1"
modelsbert = loadModel(modelsbert_path)

KeyboardInterrupt: ignored

In [ ]:
rouge1 = []
rouge2 = []
rougeL = []
rougeLsum = []
sBERT_prec = []
sBERT_recall = []
sBERT_f1 = []
BERT_prec = []
BERT_recall = []
BERT_f1 = []

for i in range(len(ground_truths)):
  if i%5==0:
    print(i)

  gs = ground_truths[i]
  ps = predicted_summary[i]

  # results_rouge = rouge.compute(predictions=[ps], references=[gs])
  results_sBERT = calcSBERTScore(modelsbert,ps,gs)
  results_sBERT_precision = results_sBERT["precision"]
  results_sBERT_recall = results_sBERT["recall"]
  results_sBERT_F1 = results_sBERT["F1"]
  # results_BERT = bertscore.compute(predictions=[ps], references=[gs], lang="en")

  # rouge1.append(results_rouge['rouge1'])
  # rouge2.append(results_rouge['rouge2'])
  # rougeL.append(results_rouge['rougeL'])
  # rougeLsum.append(results_rouge['rougeLsum'])
  sBERT_prec.append(results_sBERT_precision)
  sBERT_recall.append(results_sBERT_recall)
  sBERT_f1.append(results_sBERT_F1)
  # BERT_prec.append(results_BERT['precision'][0])
  # BERT_recall.append(results_BERT['recall'][0])
  # BERT_f1.append(results_BERT['f1'][0])
  
# df['rouge1'] = rouge1
# df['rouge2'] = rouge2
# df['rougeL'] = rougeL
# df['rougeLsum'] = rougeLsum
df['sBERT_prec'] = sBERT_prec
df['sBERT_recall'] = sBERT_recall
df['sBERT_f1'] = sBERT_f1
# df['BERT_prec'] = BERT_prec
# df['BERT_recall'] = BERT_recall
# df['BERT_f1'] = BERT_f1

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


In [ ]:
df

,machine_summary,reference_summary,relevance_scores,coherence_scores,consistency_scores,fluency_scores,sBERT,sBERT_prec,sBERT_recall,sBERT_f1
0,"donald sterling , nba team last year . sterlin...",V. Stiviano must pay back $2.6 million in gift...,0.333333,0.266667,0.200000,0.200000,"{'precision': 0.6186625, 'recall': 0.6880631, ...",0.618662,0.688063,0.651520
1,north pacific gray whale has earned a spot in ...,"The whale, Varvara, swam a round trip from Rus...",0.466667,0.266667,0.266667,0.200000,"{'precision': 0.70512533, 'recall': 0.80844474...",0.705125,0.808445,0.753259
2,russian fighter jet intercepted a u.s. reconna...,The incident occurred on April 7 north of Pola...,0.800000,0.666667,1.000000,0.733333,"{'precision': 0.6819793, 'recall': 0.7473576, ...",0.681979,0.747358,0.713173
3,michael barnett captured the fire on interstat...,Country band Lady Antebellum's bus caught fire...,0.400000,0.400000,0.466667,0.533333,"{'precision': 0.5391862, 'recall': 0.56387466,...",0.539186,0.563875,0.551254
4,deep reddish color caught seattle native tim d...,Smoke from massive fires in Siberia created fi...,0.333333,0.333333,0.400000,1.000000,"{'precision': 0.5029656, 'recall': 0.47672403,...",0.502966,0.476724,0.489493
...,...,...,...,...,...,...,...,...,...,...
95,chelsea have made an offer for fc tokyo 's dut...,Naoki Ogane says that Chelsea have made an off...,0.600000,0.400000,0.400000,0.600000,"{'precision': 0.76404095, 'recall': 0.64507455...",0.764041,0.645075,0.699536
96,christopher lawler said he was pinned to a cha...,Christopher Lawler claims he was pinned to a c...,1.000000,0.600000,1.000000,1.000000,"{'precision': 0.728815, 'recall': 0.6979325, '...",0.728815,0.697932,0.713039
97,eden hazard scored a 1-0 lead against manchest...,Eden Hazard scored the opening goal for Chelse...,0.666667,0.400000,0.333333,0.666667,"{'precision': 0.81336254, 'recall': 0.7043042,...",0.813363,0.704304,0.754915
98,evangelos patoulidis is regarded as one of the...,Evangelos Patoulidis also attracted interest f...,0.733333,0.533333,1.000000,0.866667,"{'precision': 0.6988193, 'recall': 0.634072, '...",0.698819,0.634072,0.664873


In [ ]:
df.head()

,machine_summary,reference_summary,relevance_scores,coherence_scores,consistency_scores,fluency_scores,sBERT,sBERT_prec,sBERT_recall,sBERT_f1
0,"donald sterling , nba team last year . sterlin...",V. Stiviano must pay back $2.6 million in gift...,0.333333,0.266667,0.200000,0.200000,"{'precision': 0.6186625, 'recall': 0.6880631, ...",0.618662,0.688063,0.651520
1,north pacific gray whale has earned a spot in ...,"The whale, Varvara, swam a round trip from Rus...",0.466667,0.266667,0.266667,0.200000,"{'precision': 0.70512533, 'recall': 0.80844474...",0.705125,0.808445,0.753259
2,russian fighter jet intercepted a u.s. reconna...,The incident occurred on April 7 north of Pola...,0.800000,0.666667,1.000000,0.733333,"{'precision': 0.6819793, 'recall': 0.7473576, ...",0.681979,0.747358,0.713173
3,michael barnett captured the fire on interstat...,Country band Lady Antebellum's bus caught fire...,0.400000,0.400000,0.466667,0.533333,"{'precision': 0.5391862, 'recall': 0.56387466,...",0.539186,0.563875,0.551254
4,deep reddish color caught seattle native tim d...,Smoke from massive fires in Siberia created fi...,0.333333,0.333333,0.400000,1.000000,"{'precision': 0.5029656, 'recall': 0.47672403,...",0.502966,0.476724,0.489493


In [ ]:
df.to_csv('correlation.csv')

In [ ]:
import numpy as np
from scipy import stats

In [ ]:
# corr_r1_relevance = stats.pearsonr(rouge1,relevance_scores)
# corr_r1_coherence = stats.pearsonr(rouge1,coherence_scores)
# corr_r1_consistency = stats.pearsonr(rouge1,consistency_scores)
# corr_r1_fluency = stats.pearsonr(rouge1,fluency_scores)

# corr_r2_relevance = stats.pearsonr(rouge2,relevance_scores)
# corr_r2_coherence = stats.pearsonr(rouge2,coherence_scores)
# corr_r2_consistency = stats.pearsonr(rouge2,consistency_scores)
# corr_r2_fluency = stats.pearsonr(rouge2,fluency_scores)

# corr_rL_relevance = stats.pearsonr(rougeL,relevance_scores)
# corr_rL_coherence = stats.pearsonr(rougeL,coherence_scores)
# corr_rL_consistency = stats.pearsonr(rougeL,consistency_scores)
# corr_rL_fluency = stats.pearsonr(rougeL,fluency_scores)

ValueError: ignored

In [ ]:
# print(corr_rL_relevance,corr_rL_coherence,corr_rL_consistency,corr_rL_fluency)

In [ ]:
corr_sBERT_prec_relevance = stats.pearsonr(sBERT_prec,relevance_scores)
corr_sBERT_recall_relevance = stats.pearsonr(sBERT_recall,relevance_scores)
corr_sBERT_f1_relevance = stats.pearsonr(sBERT_f1,relevance_scores)


corr_sBERT_prec_coherence = stats.pearsonr(sBERT_prec,coherence_scores)
corr_sBERT_recall_coherence = stats.pearsonr(sBERT_recall,coherence_scores)
corr_sBERT_f1_coherence = stats.pearsonr(sBERT_f1,coherence_scores)


corr_sBERT_prec_consistency = stats.pearsonr(sBERT_prec,consistency_scores)
corr_sBERT_recall_consistency = stats.pearsonr(sBERT_recall,consistency_scores)
corr_sBERT_f1_consistency = stats.pearsonr(sBERT_f1,consistency_scores)

corr_sBERT_prec_fluency = stats.pearsonr(sBERT_prec,fluency_scores)
corr_sBERT_recall_fluency = stats.pearsonr(sBERT_recall,fluency_scores)
corr_sBERT_f1_fluency = stats.pearsonr(sBERT_f1,fluency_scores)

TypeError: ignored

In [ ]:
print(corr_sBERT_relevance,corr_sBERT_coherence,corr_sBERT_consistency,corr_sBERT_fluency)

In [ ]:
corr_BERT_prec_relevance = stats.pearsonr(BERT_prec,relevance_scores)
corr_BERT_prec_coherence = stats.pearsonr(BERT_prec,coherence_scores)
corr_BERT_prec_consistency = stats.pearsonr(BERT_prec,consistency_scores)
corr_BERT_prec_fluency = stats.pearsonr(BERT_prec,fluency_scores)

corr_BERT_recall_relevance = stats.pearsonr(BERT_recall,relevance_scores)
corr_BERT_recall_coherence = stats.pearsonr(BERT_recall,coherence_scores)
corr_BERT_recall_consistency = stats.pearsonr(BERT_recall,consistency_scores)
corr_BERT_recall_fluency = stats.pearsonr(BERT_recall,fluency_scores)

corr_BERT_f1_relevance = stats.pearsonr(BERT_f1,relevance_scores)
corr_BERT_f1_coherence = stats.pearsonr(BERT_f1,coherence_scores)
corr_BERT_f1_consistency = stats.pearsonr(BERT_f1,consistency_scores)
corr_BERT_f1_fluency = stats.pearsonr(BERT_f1,fluency_scores)

In [ ]:
print(corr_BERT_prec_relevance,corr_BERT_prec_coherence,corr_BERT_prec_consistency,corr_BERT_prec_fluency)

PearsonRResult(statistic=0.34079291303413173, pvalue=0.0005213272578908398) PearsonRResult(statistic=0.32060066078795235, pvalue=0.0011460048740687644) PearsonRResult(statistic=0.17297446891230703, pvalue=0.08525161541386522) PearsonRResult(statistic=0.22560338092826443, pvalue=0.024017488163294385)


In [ ]:
print(corr_BERT_recall_relevance,corr_BERT_recall_coherence,corr_BERT_recall_consistency,corr_BERT_recall_fluency)

In [ ]:
print(corr_BERT_f1_relevance,corr_BERT_f1_coherence,corr_BERT_f1_consistency,corr_BERT_f1_fluency)

In [ ]:
corr_BERT_prec_relevance.statistic, corr_BERT_prec_relevance.statistic.pvalue,
corr_BERT_prec_coherence.statistic, corr_BERT_prec_coherence.statistic.pvalue,
corr_BERT_prec_consistency.statistic, corr_BERT_prec_consistency.pvalue,
corr_BERT_prec_fluency.statistic, corr_BERT_prec_fluency.pvalue,


In [ ]:
import pandas as pd

# Create a dictionary with the variable values
data = {
    'Metric': ['ROGUE', 'BERT', 'sBERT'],
    'Relevance': [corr_BERT_prec_relevance.statistic, corr_BERT_prec_coherence.statistic, corr_BERT_prec_consistency.statistic],
    'Coherence': [corr_BERT_prec_coherence.statistic, corr_BERT_recall_coherence.statistic, corr_BERT_f1_coherence.statistic],
    'Consistency': [corr_BERT_prec_consistency.statistic, corr_BERT_recall_consistency.statistic, corr_BERT_f1_consistency.statistic],
    'Fluency': [corr_BERT_prec_fluency.statistic, corr_BERT_recall_fluency.statistic, corr_BERT_f1_fluency.statistic]
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
df.to_excel('variable_values.xlsx', index=False)
df.to_csv('variable_values.csv', index=False)


In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
# modelsbert_path = 'paraphrase-MiniLM-L6-v2'
modelsbert_paths = ["all-mpnet-base-v2","multi-qa-mpnet-base-dot-v1","all-distilroberta-v1","all-MiniLM-L12-v2","multi-qa-distilbert-cos-v1","msmarco-distilbert-cos-v5"]


results_df = pd.DataFrame(columns=['Model', 'corr_sBERT_prec_relevance', 'corr_sBERT_recall_relevance',
                                   'corr_sBERT_f1_relevance', 'corr_sBERT_prec_coherence',
                                   'corr_sBERT_recall_coherence', 'corr_sBERT_f1_coherence',
                                   'corr_sBERT_prec_consistency', 'corr_sBERT_recall_consistency',
                                   'corr_sBERT_f1_consistency', 'corr_sBERT_prec_fluency',
                                   'corr_sBERT_recall_fluency', 'corr_sBERT_f1_fluency'])

for modelsbert_path in modelsbert_paths:
  print(modelsbert_path)
  modelsbert = loadModel(modelsbert_path)


  rouge1 = []
  rouge2 = []
  rougeL = []
  rougeLsum = []
  sBERT_prec = []
  sBERT_recall = []
  sBERT_f1 = []
  BERT_prec = []
  BERT_recall = []
  BERT_f1 = []

  for i in range(len(ground_truths)):
    if i%5==0:
      print(i)

    gs = ground_truths[i]
    ps = predicted_summary[i]

    results_rouge = rouge.compute(predictions=[ps], references=[gs])
    results_sBERT = calcSBERTScore(modelsbert,ps,gs)
    results_sBERT_precision = results_sBERT["precision"]
    results_sBERT_recall = results_sBERT["recall"]
    results_sBERT_F1 = results_sBERT["F1"]
    results_BERT = bertscore.compute(predictions=[ps], references=[gs], lang="en")

    rouge1.append(results_rouge['rouge1'])
    rouge2.append(results_rouge['rouge2'])
    rougeL.append(results_rouge['rougeL'])
    rougeLsum.append(results_rouge['rougeLsum'])
    sBERT_prec.append(results_sBERT_precision)
    sBERT_recall.append(results_sBERT_recall)
    sBERT_f1.append(results_sBERT_F1)
    BERT_prec.append(results_BERT['precision'][0])
    BERT_recall.append(results_BERT['recall'][0])
    BERT_f1.append(results_BERT['f1'][0])



  corr_sBERT_prec_relevance = stats.pearsonr(sBERT_prec,relevance_scores)
  corr_sBERT_recall_relevance = stats.pearsonr(sBERT_recall,relevance_scores)
  corr_sBERT_f1_relevance = stats.pearsonr(sBERT_f1,relevance_scores)


  corr_sBERT_prec_coherence = stats.pearsonr(sBERT_prec,coherence_scores)
  corr_sBERT_recall_coherence = stats.pearsonr(sBERT_recall,coherence_scores)
  corr_sBERT_f1_coherence = stats.pearsonr(sBERT_f1,coherence_scores)


  corr_sBERT_prec_consistency = stats.pearsonr(sBERT_prec,consistency_scores)
  corr_sBERT_recall_consistency = stats.pearsonr(sBERT_recall,consistency_scores)
  corr_sBERT_f1_consistency = stats.pearsonr(sBERT_f1,consistency_scores)

  corr_sBERT_prec_fluency = stats.pearsonr(sBERT_prec,fluency_scores)
  corr_sBERT_recall_fluency = stats.pearsonr(sBERT_recall,fluency_scores)
  corr_sBERT_f1_fluency = stats.pearsonr(sBERT_f1,fluency_scores)


  corr_rouge1_relevance = stats.pearsonr(rouge1,relevance_scores)
  corr_rouge2_relevance = stats.pearsonr(rouge2,relevance_scores)
  corr_rougeL_relevance = stats.pearsonr(rougeL,relevance_scores)
  corr_rougeLsum_relevance = stats.pearsonr(rougeLsum,relevance_scores)

  corr_rouge1_coherence = stats.pearsonr(rouge1,coherence_scores)
  corr_rouge2_coherence = stats.pearsonr(rouge2,coherence_scores)
  corr_rougeL_coherence = stats.pearsonr(rougeL,coherence_scores)
  corr_rougeLsum_coherence = stats.pearsonr(rougeLsum,coherence_scores)

  corr_rouge1_consistency = stats.pearsonr(rouge1,consistency_scores)
  corr_rouge2_consistency = stats.pearsonr(rouge2,consistency_scores)
  corr_rougeL_consistency = stats.pearsonr(rougeL,consistency_scores)
  corr_rougeLsum_consistency = stats.pearsonr(rougeLsum,consistency_scores)

  corr_rouge1_fluency = stats.pearsonr(rouge1,fluency_scores)
  corr_rouge2_fluency = stats.pearsonr(rouge2,fluency_scores)
  corr_rougeL_fluency = stats.pearsonr(rougeL,fluency_scores)
  corr_rougeLsum_fluency = stats.pearsonr(rougeLsum,fluency_scores)


  corr_BERT_prec_relevance = stats.pearsonr(BERT_prec,relevance_scores)
  corr_BERT_recall_relevance = stats.pearsonr(BERT_recall,relevance_scores)
  corr_BERT_f1_relevance = stats.pearsonr(BERT_f1,relevance_scores)

  corr_BERT_prec_coherence = stats.pearsonr(BERT_prec,coherence_scores)
  corr_BERT_recall_coherence = stats.pearsonr(BERT_recall,coherence_scores)
  corr_BERT_f1_coherence = stats.pearsonr(BERT_f1,coherence_scores)

  corr_BERT_prec_consistency = stats.pearsonr(BERT_prec,consistency_scores)
  ccorr_BERT_recall_consistency = stats.pearsonr(BERT_recall,consistency_scores)
  corr_BERT_f1_consistency = stats.pearsonr(BERT_f1,consistency_scores)

  corr_BERT_prec_fluency = stats.pearsonr(BERT_prec,fluency_scores)
  corr_BERT_recall_fluency = stats.pearsonr(BERT_recall,fluency_scores)
  corr_BERT_f1_fluency = stats.pearsonr(BERT_f1,fluency_scores)


  model_results = {
    'Model': modelsbert_path,
    'corr_sBERT_prec_relevance': corr_sBERT_prec_relevance[0],
    'corr_sBERT_recall_relevance': corr_sBERT_recall_relevance[0],
    'corr_sBERT_f1_relevance': corr_sBERT_f1_relevance[0],
    'corr_sBERT_prec_coherence': corr_sBERT_prec_coherence[0],
    'corr_sBERT_recall_coherence': corr_sBERT_recall_coherence[0],
    'corr_sBERT_f1_coherence': corr_sBERT_f1_coherence[0],
    'corr_sBERT_prec_consistency': corr_sBERT_prec_consistency[0],
    'corr_sBERT_recall_consistency': corr_sBERT_recall_consistency[0],
    'corr_sBERT_f1_consistency': corr_sBERT_f1_consistency[0],
    'corr_sBERT_prec_fluency': corr_sBERT_prec_fluency[0],
    'corr_sBERT_recall_fluency': corr_sBERT_recall_fluency[0],
    'corr_sBERT_f1_fluency': corr_sBERT_f1_fluency[0]}

  # Append the model results to the DataFrame
  results_df = results_df.append(model_results, ignore_index=True)












results_df.to_csv("sbert_results.csv", index=False)

all-mpnet-base-v2


0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


<ipython-input-25-8a5a05a1075e>:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(model_results, ignore_index=True)


multi-qa-mpnet-base-dot-v1
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


<ipython-input-25-8a5a05a1075e>:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(model_results, ignore_index=True)


...

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


<ipython-input-25-8a5a05a1075e>:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(model_results, ignore_index=True)


msmarco-distilbert-cos-v5


0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


<ipython-input-25-8a5a05a1075e>:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(model_results, ignore_index=True)


In [ ]:
results_df

,Model,corr_sBERT_prec_relevance,corr_sBERT_recall_relevance,corr_sBERT_f1_relevance,corr_sBERT_prec_coherence,corr_sBERT_recall_coherence,corr_sBERT_f1_coherence,corr_sBERT_prec_consistency,corr_sBERT_recall_consistency,corr_sBERT_f1_consistency,corr_sBERT_prec_fluency,corr_sBERT_recall_fluency,corr_sBERT_f1_fluency
0,multi-qa-mpnet-base-dot-v1,0.189458,0.118094,0.171959,0.244771,0.037874,0.161535,0.076629,-0.011989,0.036514,0.265883,0.164525,0.238999


In [ ]:
!pip install rouge-score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

In [ ]:
import pandas as pd
from scipy import stats

results_df = pd.DataFrame(columns=['Model', 'corr_relevance', 'corr_coherence', 'corr_consistency', 'corr_fluency'])


for i in range(len(ground_truths)):
    rouge1 = []
    rouge2 = []
    rougeL = []
    rougeLsum = []
    BERT_prec = []
    BERT_recall = []
    BERT_f1 = []


    if i % 5 == 0:
        print(i)

    gs = ground_truths[i]
    ps = predicted_summary[i]

    results_rouge = rouge.compute(predictions=[ps], references=[gs])
    rouge1.append(results_rouge['rouge1'])
    rouge2.append(results_rouge['rouge2'])
    rougeL.append(results_rouge['rougeL'])
    rougeLsum.append(results_rouge['rougeLsum'])

    results_BERT = bertscore.compute(predictions=[ps], references=[gs], lang="en")
    BERT_prec.append(results_BERT['precision'][0])
    BERT_recall.append(results_BERT['recall'][0])
    BERT_f1.append(results_BERT['f1'][0])


corr_rouge1_relevance = stats.pearsonr(rouge1, relevance_scores)
corr_rouge2_relevance = stats.pearsonr(rouge2, relevance_scores)
corr_rougeL_relevance = stats.pearsonr(rougeL, relevance_scores)
corr_rougeLsum_relevance = stats.pearsonr(rougeLsum, relevance_scores)

corr_rouge1_coherence = stats.pearsonr(rouge1, coherence_scores)
corr_rouge2_coherence = stats.pearsonr(rouge2, coherence_scores)
corr_rougeL_coherence = stats.pearsonr(rougeL, coherence_scores)
corr_rougeLsum_coherence = stats.pearsonr(rougeLsum, coherence_scores)

corr_rouge1_consistency = stats.pearsonr(rouge1, consistency_scores)
corr_rouge2_consistency = stats.pearsonr(rouge2, consistency_scores)
corr_rougeL_consistency = stats.pearsonr(rougeL, consistency_scores)
corr_rougeLsum_consistency = stats.pearsonr(rougeLsum, consistency_scores)

corr_rouge1_fluency = stats.pearsonr(rouge1, fluency_scores)
corr_rouge2_fluency = stats.pearsonr(rouge2, fluency_scores)
corr_rougeL_fluency = stats.pearsonr(rougeL, fluency_scores)
corr_rougeLsum_fluency = stats.pearsonr(rougeLsum, fluency_scores)

corr_BERT_prec_relevance = stats.pearsonr(BERT_prec, relevance_scores)
corr_BERT_recall_relevance = stats.pearsonr(BERT_recall, relevance_scores)
corr_BERT_f1_relevance = stats.pearsonr(BERT_f1, relevance_scores)

corr_BERT_prec_coherence = stats.pearsonr(BERT_prec, coherence_scores)
corr_BERT_recall_coherence = stats.pearsonr(BERT_recall, coherence_scores)
corr_BERT_f1_coherence = stats.pearsonr(BERT_f1, coherence_scores)

corr_BERT_prec_consistency = stats.pearsonr(BERT_prec, consistency_scores)
corr_BERT_recall_consistency = stats.pearsonr(BERT_recall, consistency_scores)
corr_BERT_f1_consistency = stats.pearsonr(BERT_f1, consistency_scores)

corr_BERT_prec_fluency = stats.pearsonr(BERT_prec, fluency_scores)
corr_BERT_recall_fluency = stats.pearsonr(BERT_recall, fluency_scores)
corr_BERT_f1_fluency = stats.pearsonr(BERT_f1, fluency_scores)



results_df = results_df.append({'Model': 'BERT_prec',
                                'corr_relevance': corr_BERT_prec_relevance,
                                'corr_coherence': corr_BERT_prec_coherence,
                                'corr_consistency': corr_BERT_prec_consistency,
                                'corr_fluency': corr_BERT_prec_fluency}, ignore_index=True)

results_df = results_df.append({'Model': 'BERT_recall',
                                'corr_relevance': corr_BERT_recall_relevance,
                                'corr_coherence': corr_BERT_recall_coherence,
                                'corr_consistency': corr_BERT_recall_consistency,
                                'corr_fluency': corr_BERT_recall_fluency}, ignore_index=True)

results_df = results_df.append({'Model': 'BERT_f1',
                                'corr_relevance': corr_BERT_f1_relevance,
                                'corr_coherence': corr_BERT_f1_coherence,
                                'corr_consistency': corr_BERT_f1_consistency,
                                'corr_fluency': corr_BERT_f1_fluency}, ignore_index=True)


results_df = results_df.append({'Model': 'rouge1',
                                'corr_relevance': corr_rouge1_relevance,
                                'corr_coherence': corr_rouge1_coherence,
                                'corr_consistency': corr_rouge1_consistency,
                                'corr_fluency': corr_rouge1_fluency}, ignore_index=True)

results_df = results_df.append({'Model': 'rouge2',
                                'corr_relevance': corr_rouge2_relevance,
                                'corr_coherence': corr_rouge2_coherence,
                                'corr_consistency': corr_rouge2_consistency,
                                'corr_fluency': corr_rouge2_fluency}, ignore_index=True)

results_df = results_df.append({'Model': 'rougeL',
                                'corr_relevance': corr_rougeL_relevance,
                                'corr_coherence': corr_rougeL_coherence,
                                'corr_consistency': corr_rougeL_consistency,
                                'corr_fluency': corr_rougeL_fluency}, ignore_index=True)

results_df = results_df.append({'Model': 'rougeLsum',
                                'corr_relevance': corr_rougeLsum_relevance,
                                'corr_coherence': corr_rougeLsum_coherence,
                                'corr_consistency': corr_rougeLsum_consistency,
                                'corr_fluency': corr_rougeLsum_fluency}, ignore_index=True)



0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


ValueError: ignored

In [ ]:
modelsbert_paths = ["all-mpnet-base-v2", "multi-qa-mpnet-base-dot-v1", "all-distilroberta-v1",
                    "all-MiniLM-L12-v2", "multi-qa-distilbert-cos-v1", "msmarco-distilbert-cos-v5"]


for modelsbert_path in modelsbert_paths:
    print(modelsbert_path)
    modelsbert = loadModel(modelsbert_path)


    sBERT_prec = []
    sBERT_recall = []
    sBERT_f1 = []

    for i in range(len(ground_truths)):
        if i % 5 == 0:
            print(i)

        gs = ground_truths[i]
        ps = predicted_summary[i]


        results_sBERT = calcSBERTScore(modelsbert, ps, gs)
        results_sBERT_precision = results_sBERT["precision"]
        results_sBERT_recall = results_sBERT["recall"]
        results_sBERT_F1 = results_sBERT["F1"]

        sBERT_prec.append(results_sBERT_precision)
        sBERT_recall.append(results_sBERT_recall)
        sBERT_f1.append(results_sBERT_F1)

    corr_sBERT_prec_relevance = stats.pearsonr(sBERT_prec, relevance_scores)
    corr_sBERT_recall_relevance = stats.pearsonr(sBERT_recall, relevance_scores)
    corr_sBERT_f1_relevance = stats.pearsonr(sBERT_f1, relevance_scores)

    corr_sBERT_prec_coherence = stats.pearsonr(sBERT_prec, coherence_scores)
    corr_sBERT_recall_coherence = stats.pearsonr(sBERT_recall, coherence_scores)
    corr_sBERT_f1_coherence = stats.pearsonr(sBERT_f1, coherence_scores)

    corr_sBERT_prec_consistency = stats.pearsonr(sBERT_prec, consistency_scores)
    corr_sBERT_recall_consistency = stats.pearsonr(sBERT_recall, consistency_scores)
    corr_sBERT_f1_consistency = stats.pearsonr(sBERT_f1, consistency_scores)

    corr_sBERT_prec_fluency = stats.pearsonr(sBERT_prec, fluency_scores)
    corr_sBERT_recall_fluency = stats.pearsonr(sBERT_recall, fluency_scores)
    corr_sBERT_f1_fluency = stats.pearsonr(sBERT_f1, fluency_scores)



    results_df = results_df.append({'Model': f'{modelsbert_path}_prec',
                                    'corr_relevance': corr_sBERT_prec_relevance[0],
                                    'corr_coherence': corr_sBERT_prec_coherence[0],
                                    'corr_consistency': corr_sBERT_prec_consistency[0],
                                    'corr_fluency': corr_sBERT_prec_fluency[0]}, ignore_index=True)

    results_df = results_df.append({'Model': f'{modelsbert_path}_recall',
                                    'corr_relevance': corr_sBERT_recall_relevance[0],
                                    'corr_coherence': corr_sBERT_recall_coherence[0],
                                    'corr_consistency': corr_sBERT_recall_consistency[0],
                                    'corr_fluency': corr_sBERT_recall_fluency}, ignore_index=True)

    results_df = results_df.append({'Model': f'{modelsbert_path}_f1',
                                    'corr_relevance': corr_sBERT_f1_relevance[0],
                                    'corr_coherence': corr_sBERT_f1_coherence[0],
                                    'corr_consistency': corr_sBERT_f1_consistency[0],
                                    'corr_fluency': corr_sBERT_f1_fluency[0]}, ignore_index=True)



# Save the results to a CSV file
results_df.to_csv('correlation_results.csv', index=False)